# Debug Tmobile sjoin + overlay

`overlay` is taking a long time, but since there's a dissolve, it's basically 1 row for T-Mobile compared to 1 row for the district boundary. Might be able to treat them as GeoSeries and see if it can go through faster.

`difference` can also take multiple rows for T-Mobile compared to 1 row for district boundary. In this case, the dissolve for where there's no coverage for T-Mobile should take place after the difference is taken.

* `difference`: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.difference.html


In [ ]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import pandas as pd
from shared_utils import geography_utils
from shared_utils import utils

from calitp.sql import to_snakecase
import utilities

In [ ]:
counties = utilities.get_counties()

# Read in original CT map
ct_all_districts = to_snakecase(
        gpd.read_file(f"{utilities.caltrans_shape}").to_crs(epsg=4326)
    )[["district", "geometry"]]
    

##### Sjoin  for T-Mobile by Districts
* Do a sjoin to only grab rows of T-Mobile's gdf that overlap with Caltrans districts before `find difference and clip`.
* Counties didn't seem to work, that's why I went with districts.
* Done: Results are split between 3 parquets.
    * `gs://calitp-analytics-data/data-analyses/cellular_coverage/tmobile_d1_d4.parquet` 
    * `gs://calitp-analytics-data/data-analyses/cellular_coverage/tmobile_d5_d8.parquet`
    * `gs://calitp-analytics-data/data-analyses/cellular_coverage/tmobile_d9_d12.parquet`
    

* Took about 3 minutes to finish running.

In [ ]:
# Original map - includes other stuff that isn't California
tmobile = dg.read_parquet(
    "gs://calitp-analytics-data/data-analyses/cellular_coverage/tmobile_california.parquet"
)

In [ ]:
# Blue parts are areas with coverage in D5. 
tmobile_district = dg.sjoin(
    tmobile[["geometry"]], 
    ct_all_districts[ct_all_districts.district==5],
    how="inner", 
    predicate="intersects"
).drop(columns = "index_right")

In [ ]:
tmobile_d5 = tmobile_district.compute()

In [ ]:
len(tmobile_d5)

In [ ]:
tmobile_d5.plot()

In [ ]:
# Getting areas without coverage in D2. 
no_coverage = tmobile_d5.difference(
        ct_all_districts[ct_all_districts.district==5].geometry.iloc[0], 
    ).reset_index()

In [ ]:
# This line grabs the actual polygon 
ct_all_districts[ct_all_districts.district==2].geometry.iloc[0]

In [ ]:
no_coverage.head()

In [ ]:
 len(no_coverage)

In [ ]:
no_coverage_d5 = (no_coverage.reset_index()
                  .dissolve()
                  .rename(columns = {0: 'geometry'})
                  [["geometry"]]
                 )

In [ ]:
# type(no_coverage_d2)

In [ ]:
no_coverage_d5 = no_coverage_d5.set_geometry('geometry')

In [ ]:
no_coverage_d5.plot()

#### Break apart dissolve

In [ ]:
ct_d5  = (ct_all_districts[ct_all_districts.district==5]

In [ ]:
ct_d5

In [ ]:
ct_all_districts[ct_all_districts.district==5]

In [ ]:
ct_d5.plot()

In [ ]:
tmobile_district = dg.sjoin(
        tmobile[['geometry']], 
        ct_d5, 
        how="inner", 
        predicate="intersects"
    )

In [ ]:
tmobile_district_gdf = tmobile_district.compute()

In [ ]:
len(tmobile_district_gdf)

In [ ]:
tmobile_district_gdf.head(1)

In [ ]:
"""
tmobile_5 = (tmobile_district.drop(columns = "index_right").dissolve(by="district")
                    .reset_index()
                    .compute()
                   )"""

In [ ]:
len(tmobile_5)

In [ ]:
district_file = tmobile_5.district.iloc[0]

In [ ]:
district_file

In [ ]:
tmobile_5

In [ ]:
# utils.geoparquet_gcs_export(tmobile_5, utilities.GCS_FILE_PATH,f"tmobile_d{district_file}.parquet")

In [ ]:
tmobile_align_true = (tmobile_5.difference(
            (ct_all_districts[ct_all_districts.district==5]).reset_index(), 
            align=True
        ).reset_index()
        .rename(columns = {0: 'geometry'})
    )   

In [ ]:
type(tmobile_align_true)

In [ ]:
tmobile_align_true = tmobile_align_true.set_geometry('geometry')

In [ ]:
# I get an error when plotting /opt/conda/lib/python3.9/site-packages/geopandas/array.py:938: RuntimeWarning: All-NaN slice encountered
tmobile_align_true.plot()

In [ ]:
def sjoin_to_district_find_difference(
    tmobile: dg.GeoDataFrame, 
    district_df: gpd.GeoDataFrame
) -> dg.GeoDataFrame:
    tmobile_district = dg.sjoin(
        tmobile, 
        district_df, 
        how="inner", 
        predicate="intersects"
    ).drop(columns = "index_right")
    
    # Dissolve so that it's just 1 row for TMobile coverage in that district
    tmobile_diss = (tmobile_district.dissolve(by="district")
                    .reset_index()
                    .compute()
                   )
    
    # Maybe stash intermediate output here?
    d = district_df.district.iloc[0]
    tmobile_diss.to_parquet(f"tmobile_d{d}.parquet")
    
    # Instead of gpd.overlay(), since tmobile_diss is 1 row and 
    # district_df is 1 row,
    # can treat them as GeoSeries, align=True, to do row-wise comparison
    no_coverage = (tmobile_diss.difference(
            ct_all_districts[ct_all_districts.district==i], 
            align=True
        ).reset_index()
        .rename(columns = {0: 'geometry'})
    )   
    
    no_coverage.to_parquet(f"no_coverage_d{d}.parquet")
    
    return no_coverage

In [ ]:
results = []

for i in [1]:
    dissolved_result = sjoin_to_district_find_difference(
        tmobile[["geometry"]], 
        ct_all_districts[ct_all_districts.district==i]
    )
    
    results.append(dissolved_result)

In [ ]:
ddf = dd.multi.concat(results, axis=0)

In [ ]:
df = ddf.compute()